In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns 
from rdkit import Chem
import mols2grid
import torch
import rdkit
from tqdm import tqdm

In [2]:
dw = torch.load("/scratch/dz1061/pkasolver-data/exp_processed_all.pt")

In [3]:
dw.head()

,exp_id,pka,center,prot_smiles,deprot_smiles,prot_charge,deprot_charge,prot_mol,deprot_mol,acid_base_type,acid_base_check,acid_base_string,ROMol,MolWt,HeavyAtomCount,NumHDonors,NumHAcceptors,MolLogP,NumRotatableBonds,SMILES
0,mol0,6.21,10,Brc1c(N2CCCCCC2)nc(C2CC2)[nH+]c1NC1CC1,Brc1c(NC2CC2)nc(C2CC2)nc1N1CCCCCC1,1,0,<rdkit.Chem.PropertyMol.PropertyMol object at ...,<rdkit.Chem.PropertyMol.PropertyMol object at ...,basic,consistent,B,<rdkit.Chem.PropertyMol.PropertyMol object at ...,350.110609,21,1,4,4.0712,4,Brc1c(NC2CC2)nc(C2CC2)nc1N1CCCCCC1
1,mol1,7.46,9,Brc1cc(Br)c(NC2=[NH+]CCN2)c(Br)c1,Brc1cc(Br)c(NC2=NCCN2)c(Br)c1,1,0,<rdkit.Chem.PropertyMol.PropertyMol object at ...,<rdkit.Chem.PropertyMol.PropertyMol object at ...,basic,consistent,B,<rdkit.Chem.PropertyMol.PropertyMol object at ...,394.826834,15,2,3,3.3452,1,Brc1cc(Br)c(NC2=NCCN2)c(Br)c1
2,mol2,4.2,9,Brc1cc2cccnc2c2[nH+]cccc12,Brc1cc2cccnc2c2ncccc12,1,0,<rdkit.Chem.PropertyMol.PropertyMol object at ...,<rdkit.Chem.PropertyMol.PropertyMol object at ...,basic,consistent,B,<rdkit.Chem.PropertyMol.PropertyMol object at ...,257.979260,15,0,2,3.5455,0,Brc1cc2cccnc2c2ncccc12
3,mol3,3.73,8,Brc1ccc(-c2nn[nH]n2)cc1,Brc1ccc(-c2nn[n-]n2)cc1,0,-1,<rdkit.Chem.PropertyMol.PropertyMol object at ...,<rdkit.Chem.PropertyMol.PropertyMol object at ...,acidic,consistent,A,<rdkit.Chem.PropertyMol.PropertyMol object at ...,223.969758,12,1,3,1.6292,1,Brc1ccc(-c2nn[nH]n2)cc1
4,mol4,11.0,13,Brc1ccc(Br)c(N(CC2CC2)C2=[NH+]CCN2)c1,Brc1ccc(Br)c(N(CC2CC2)C2=NCCN2)c1,1,0,<rdkit.Chem.PropertyMol.PropertyMol object at ...,<rdkit.Chem.PropertyMol.PropertyMol object at ...,basic,consistent,B,<rdkit.Chem.PropertyMol.PropertyMol object at ...,370.963272,18,1,3,3.3872,3,Brc1ccc(Br)c(N(CC2CC2)C2=NCCN2)c1


In [62]:
dw.shape

(5994, 20)

In [47]:
dw.groupby('acid_base_string').count()

,exp_id,pka,center,prot_smiles,deprot_smiles,prot_charge,deprot_charge,prot_mol,deprot_mol,acid_base_type,acid_base_check,ROMol,MolWt,HeavyAtomCount,NumHDonors,NumHAcceptors,MolLogP,NumRotatableBonds,SMILES
acid_base_string,,,,,,,,,,,,,,,,,,,
A,2107,2107,2107,2107,2107,2107,2107,2107,2107,2107,2107,2107,2107,2107,2107,2107,2107,2107,2107
B,3887,3887,3887,3887,3887,3887,3887,3887,3887,3887,3887,3887,3887,3887,3887,3887,3887,3887,3887


In [48]:
dw.groupby('acid_base_type').count()

,exp_id,pka,center,prot_smiles,deprot_smiles,prot_charge,deprot_charge,prot_mol,deprot_mol,acid_base_check,acid_base_string,ROMol,MolWt,HeavyAtomCount,NumHDonors,NumHAcceptors,MolLogP,NumRotatableBonds,SMILES
acid_base_type,,,,,,,,,,,,,,,,,,,
acidic,2398,2398,2398,2398,2398,2398,2398,2398,2398,2398,2398,2398,2398,2398,2398,2398,2398,2398,2398
basic,3596,3596,3596,3596,3596,3596,3596,3596,3596,3596,3596,3596,3596,3596,3596,3596,3596,3596,3596


In [2]:
novartis = torch.load("/scratch/dz1061/pkasolver-data/novartis_processed_all.pt")

In [66]:
novartis

,exp_id,pka,center,prot_smiles,deprot_smiles,prot_charge,deprot_charge,prot_mol,deprot_mol,acid_base_type,acid_base_check,acid_base_string,ROMol,MolWt,HeavyAtomCount,NumHDonors,NumHAcceptors,MolLogP,NumRotatableBonds,SMILES
0,mol0,8.55,13,Brc1ccc(C2CN3C=CSC3=[NH+]2)cc1,Brc1ccc(C2CN3C=CSC3=N2)cc1,1,0,<rdkit.Chem.PropertyMol.PropertyMol object at ...,<rdkit.Chem.PropertyMol.PropertyMol object at ...,basic,consistent,B,<rdkit.Chem.PropertyMol.PropertyMol object at ...,279.966981,15,0,3,3.37980,1,Brc1ccc(C2CN3C=CSC3=N2)cc1
1,mol1,5.5,4,C/C(=C\c1cc(C)cc[nH+]1)[C@@H]1C[C@@H]2O[C@]2(C...,C/C(=C\c1cc(C)ccn1)[C@@H]1C[C@@H]2O[C@]2(C)CCC...,1,0,<rdkit.Chem.PropertyMol.PropertyMol object at ...,<rdkit.Chem.PropertyMol.PropertyMol object at ...,basic,consistent,B,<rdkit.Chem.PropertyMol.PropertyMol object at ...,501.309038,36,2,7,4.41602,2,C/C(=C\c1cc(C)ccn1)[C@@H]1C[C@@H]2O[C@]2(C)CCC...
2,mol2,4.5,5,C/C(=C\c1ccc(C)c[nH+]1)[C@@H]1C[C@@H]2O[C@]2(C...,C/C(=C\c1ccc(C)cn1)[C@@H]1C[C@@H]2O[C@]2(C)CCC...,1,0,<rdkit.Chem.PropertyMol.PropertyMol object at ...,<rdkit.Chem.PropertyMol.PropertyMol object at ...,basic,consistent,B,<rdkit.Chem.PropertyMol.PropertyMol object at ...,501.309038,36,2,7,4.41602,2,C/C(=C\c1ccc(C)cn1)[C@@H]1C[C@@H]2O[C@]2(C)CCC...
3,mol3,5.5,6,C/C(=C\c1cccc(C)[nH+]1)[C@@H]1C[C@@H]2O[C@]2(C...,C/C(=C\c1cccc(C)n1)[C@@H]1C[C@@H]2O[C@]2(C)CCC...,1,0,<rdkit.Chem.PropertyMol.PropertyMol object at ...,<rdkit.Chem.PropertyMol.PropertyMol object at ...,basic,consistent,B,<rdkit.Chem.PropertyMol.PropertyMol object at ...,501.309038,36,2,7,4.41602,2,C/C(=C\c1cccc(C)n1)[C@@H]1C[C@@H]2O[C@]2(C)CCC...
4,mol4,5.0,32,C/C(=C\c1cccc[nH+]1)[C@@H]1C[C@@H]2O[C@]2(C)CC...,C/C(=C\c1ccccn1)[C@@H]1C[C@@H]2O[C@]2(C)CCC[C@...,1,0,<rdkit.Chem.PropertyMol.PropertyMol object at ...,<rdkit.Chem.PropertyMol.PropertyMol object at ...,basic,consistent,B,<rdkit.Chem.PropertyMol.PropertyMol object at ...,487.293388,35,2,7,4.10760,2,C/C(=C\c1ccccn1)[C@@H]1C[C@@H]2O[C@]2(C)CCC[C@...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,mol275,4.9,18,c1ccc(-c2cnc3c(-c4cc[nH+]c5ccccc45)cnn3c2)cc1,c1ccc(-c2cnc3c(-c4ccnc5ccccc45)cnn3c2)cc1,1,0,<rdkit.Chem.PropertyMol.PropertyMol object at ...,<rdkit.Chem.PropertyMol.PropertyMol object at ...,basic,consistent,B,<rdkit.Chem.PropertyMol.PropertyMol object at ...,322.121846,25,0,4,4.61150,2,c1ccc(-c2cnc3c(-c4ccnc5ccccc45)cnn3c2)cc1
276,mol276,4.8,10,c1ccc(-c2[nH+]c3n(c2-c2ccccc2)CCS3)cc1,c1ccc(-c2nc3n(c2-c2ccccc2)CCS3)cc1,1,0,<rdkit.Chem.PropertyMol.PropertyMol object at ...,<rdkit.Chem.PropertyMol.PropertyMol object at ...,basic,consistent,B,<rdkit.Chem.PropertyMol.PropertyMol object at ...,278.087769,20,0,3,4.32280,2,c1ccc(-c2nc3n(c2-c2ccccc2)CCS3)cc1
277,mol277,9.0,9,c1ccc(N2CC[NH2+]CC2)cc1,c1ccc(N2CCNCC2)cc1,1,0,<rdkit.Chem.PropertyMol.PropertyMol object at ...,<rdkit.Chem.PropertyMol.PropertyMol object at ...,basic,consistent,B,<rdkit.Chem.PropertyMol.PropertyMol object at ...,162.115698,12,1,2,1.09620,1,c1ccc(N2CCNCC2)cc1
278,mol278,8.0,14,c1ccc(Nc2nc(NC3CCCCC3)[nH+]c3ccccc23)cc1,c1ccc(Nc2nc(NC3CCCCC3)nc3ccccc23)cc1,1,0,<rdkit.Chem.rdchem.RWMol object at 0x15023543c...,<rdkit.Chem.rdchem.Mol object at 0x150236f22b80>,basic,consistent,B,<rdkit.Chem.rdchem.Mol object at 0x150236f22b80>,318.184447,24,2,4,5.11800,4,c1ccc(N=c2[nH]c(=NC3CCCCC3)[nH]c3ccccc23)cc1


In [65]:
novartis.shape

(280, 20)

In [13]:
novartis.groupby('acid_base_string').count()

,exp_id,pka,center,prot_smiles,deprot_smiles,prot_charge,deprot_charge,prot_mol,deprot_mol,acid_base_type,acid_base_check,ROMol,SMILES
acid_base_string,,,,,,,,,,,,,
A,95,95,95,95,95,95,95,95,95,95,95,95,95
B,185,185,185,185,185,185,185,185,185,185,185,185,185


In [14]:
novartis.groupby('acid_base_type').count()

,exp_id,pka,center,prot_smiles,deprot_smiles,prot_charge,deprot_charge,prot_mol,deprot_mol,acid_base_check,acid_base_string,ROMol,SMILES
acid_base_type,,,,,,,,,,,,,
acidic,112,112,112,112,112,112,112,112,112,112,112,112,112
basic,168,168,168,168,168,168,168,168,168,168,168,168,168


In [3]:
liter = torch.load("/scratch/dz1061/pkasolver-data/literature_processed.pt")

In [67]:
liter

,exp_id,pka,center,prot_smiles,deprot_smiles,prot_charge,deprot_charge,prot_mol,deprot_mol,acid_base_type,acid_base_string,acid_base_check,ROMol,SMILES
0,mol0,7.42,10,C#CC[NH+](C)[C@H](C)Cc1ccc(F)cc1,C#CCN(C)[C@H](C)Cc1ccc(F)cc1,1,0,<rdkit.Chem.PropertyMol.PropertyMol object at ...,<rdkit.Chem.PropertyMol.PropertyMol object at ...,basic,B,consistent,<rdkit.Chem.PropertyMol.PropertyMol object at ...,C#CCN(C)[C@H](C)Cc1ccc(F)cc1
1,mol1,9.50,9,CC(=O)CC1CCCC[NH2+]1,CC(=O)CC1CCCCN1,1,0,<rdkit.Chem.PropertyMol.PropertyMol object at ...,<rdkit.Chem.PropertyMol.PropertyMol object at ...,basic,B,consistent,<rdkit.Chem.PropertyMol.PropertyMol object at ...,CC(=O)CC1CCCCN1
2,mol2,9.10,6,CC(=O)CCC[NH+](C)C,CC(=O)CCCN(C)C,1,0,<rdkit.Chem.PropertyMol.PropertyMol object at ...,<rdkit.Chem.PropertyMol.PropertyMol object at ...,basic,B,consistent,<rdkit.Chem.PropertyMol.PropertyMol object at ...,CC(=O)CCCN(C)C
3,mol3,3.60,8,CC(=O)OC(CC(=O)O)C[N+](C)(C)C,CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C,1,0,<rdkit.Chem.PropertyMol.PropertyMol object at ...,<rdkit.Chem.PropertyMol.PropertyMol object at ...,acidic,A,consistent,<rdkit.Chem.PropertyMol.PropertyMol object at ...,CC(=O)OC(CC(=O)O)C[N+](C)(C)C
4,mol4,9.40,8,CC(=O)OCCCC[NH+](C)C,CC(=O)OCCCCN(C)C,1,0,<rdkit.Chem.PropertyMol.PropertyMol object at ...,<rdkit.Chem.PropertyMol.PropertyMol object at ...,basic,B,consistent,<rdkit.Chem.PropertyMol.PropertyMol object at ...,CC(=O)OCCCCN(C)C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,mol118,9.90,10,[NH3+]CCCSc1ccccc1,NCCCSc1ccccc1,1,0,<rdkit.Chem.PropertyMol.PropertyMol object at ...,<rdkit.Chem.PropertyMol.PropertyMol object at ...,basic,B,consistent,<rdkit.Chem.PropertyMol.PropertyMol object at ...,NCCCSc1ccccc1
119,mol119,9.00,9,[NH3+]CCSc1ccccc1,NCCSc1ccccc1,1,0,<rdkit.Chem.PropertyMol.PropertyMol object at ...,<rdkit.Chem.PropertyMol.PropertyMol object at ...,basic,B,consistent,<rdkit.Chem.PropertyMol.PropertyMol object at ...,NCCSc1ccccc1
120,mol120,8.37,17,[NH3+]Cc1cc(-c2ccccc2)nc2ccccc12,NCc1cc(-c2ccccc2)nc2ccccc12,1,0,<rdkit.Chem.PropertyMol.PropertyMol object at ...,<rdkit.Chem.PropertyMol.PropertyMol object at ...,basic,B,consistent,<rdkit.Chem.PropertyMol.PropertyMol object at ...,NCc1cc(-c2ccccc2)nc2ccccc12
121,mol121,9.00,9,c1ccc(N2CC[NH2+]CC2)cc1,c1ccc(N2CCNCC2)cc1,1,0,<rdkit.Chem.PropertyMol.PropertyMol object at ...,<rdkit.Chem.PropertyMol.PropertyMol object at ...,basic,B,consistent,<rdkit.Chem.PropertyMol.PropertyMol object at ...,c1ccc(N2CCNCC2)cc1


In [16]:
liter.groupby('acid_base_string').count()

,exp_id,pka,center,prot_smiles,deprot_smiles,prot_charge,deprot_charge,prot_mol,deprot_mol,acid_base_type,acid_base_check,ROMol,SMILES
acid_base_string,,,,,,,,,,,,,
A,28,28,28,28,28,28,28,28,28,28,28,28,28
B,95,95,95,95,95,95,95,95,95,95,95,95,95


In [17]:
liter.groupby('acid_base_type').count()

,exp_id,pka,center,prot_smiles,deprot_smiles,prot_charge,deprot_charge,prot_mol,deprot_mol,acid_base_string,acid_base_check,ROMol,SMILES
acid_base_type,,,,,,,,,,,,,
acidic,26,26,26,26,26,26,26,26,26,26,26,26,26
basic,97,97,97,97,97,97,97,97,97,97,97,97,97


In [14]:
from collections import defaultdict

from rdkit import Chem

def getEleSpread(smiList):
    element_mol = defaultdict(list)
    element = set()
    for idx, smi in enumerate(smiList):
        mol = smi
        #mol = Chem.MolFromSmiles(smi)
        if mol:
            for c in set([atom.GetSymbol() for atom in mol.GetAtoms()]):
                #if c not in element_mol:
                    element_mol[c].append(smi)
                #element.add(c)
            #break
    return {i:len(item) for i, item in element_mol.items()}

In [19]:
getEleSpread(dw['ROMol'])

{'C': 5994,
 'N': 4958,
 'Br': 224,
 'S': 1047,
 'O': 4371,
 'Cl': 698,
 'F': 584,
 'P': 34,
 'I': 49,
 'As': 2,
 'H': 1}

In [21]:
dw_use = dw.loc[~dw['prot_smiles'].str.contains('As'), :]

In [22]:
getEleSpread(novartis['ROMol'])

{'C': 280, 'N': 266, 'S': 77, 'Br': 10, 'O': 237, 'F': 65, 'Cl': 49, 'I': 1}

In [23]:
getEleSpread(liter['ROMol'])

{'C': 123, 'N': 110, 'F': 17, 'O': 91, 'Cl': 8, 'S': 21, 'Br': 1}

In [15]:
getEleSpread(sampl6['ROMol'])

{'O': 47, 'N': 71, 'C': 71, 'F': 10, 'S': 16, 'Cl': 16, 'Br': 6, 'I': 2}

In [5]:
import sys
sys.path.append("/scratch/dz1061/jupyternotebook/pka/src/scripts")

from featurization import *

In [6]:
def featurize(df, emb_path):
    examples = []
    smi_rep_all = torch.load(f'{emb_path}/SMILES_repr_all.pt')
    for i in tqdm(df.index):
        #print(i)
        mol, pka, center, type_, smiles = df.loc[i, 'ROMol'], df.loc[i, 'pka'], df.loc[i, 'center'],df.loc[i, 'acid_base_string'],df.loc[i, 'SMILES']
        molgraphs = {}

        mol_graph = MolGraph(mol) # 2d or 3d
        molgraphs['x'] = torch.FloatTensor(mol_graph.f_atoms) # 2d
        molgraphs['robert_x'] = smi_rep_all[smiles]

        atomic_number = []
        for atom in mol.GetAtoms():
            atomic_number.append(atom.GetAtomicNum())
        z = torch.tensor(atomic_number, dtype=torch.long)
        molgraphs['Z'] = z
        molgraphs['N'] = torch.FloatTensor([mol.GetNumAtoms()])

        mask = [0] * mol.GetNumAtoms() 
        mask[int(center)] = 1

        molgraphs['edge_attr'] = torch.FloatTensor(mol_graph.f_bonds)
        molgraphs['edge_index'] = torch.LongTensor(np.concatenate([mol_graph.at_begin, mol_graph.at_end]).reshape(2,-1))
        molgraphs['mol_y'] = torch.FloatTensor([float(pka)])
        molgraphs['mask'] = torch.FloatTensor(mask)


        examples.append(molgraphs)
        #break
        #break
    return examples

In [55]:
dw_examples = featurize(dw, '/scratch/dz1061/jupyternotebook/pka/exp/SMILES/')

100%|██████████| 5994/5994 [00:06<00:00, 889.17it/s] 


In [56]:
dw_examples[0]

{'x': tensor([[0., 1., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 'robert_x': tensor([[-2.3176e-01,  0.0000e+00,  9.3405e-03, -1.0459e-01, -1.5441e-01,
          -6.4354e-02,  2.3519e-01, -1.6352e-01, -5.0831e-01,  4.7835e-02,
          -2.2275e-01,  6.1566e-01,  2.1720e+00, -1.1405e+00,  2.3276e-02,
           4.7442e+00,  3.6033e+00, -5.3430e-01,  1.7626e-01, -2.8896e+00,
          -9.3898e+00, -7.1846e+00,  4.1523e+00, -6.2732e+00,  1.2341e-01,
          -4.7394e+00,  3.2554e+00,  1.0254e+01, -3.5801e-01, -2.4470e-01,
          -5.4949e-01,  8.6826e+00, -6.2036e+00, -5.9112e-02,  2.3402e+00,
           6.0443e-02,  5.8363e-02,  1.1708e-01,  7.5346e+00, -2.1624e-01,
          -2.4504e-01, -6.3775e-01,  9.8319e+00, -1.5700e+00,  1.9236e+01,
           1.7847e+00, -4.1605e-01,  6.9584e-0

In [68]:
dw_acid = [dw_examples[i] for i in dw[dw['acid_base_string'] == 'A'].index]

In [69]:
len(dw_acid)

2107

In [70]:
dw_base = [dw_examples[i] for i in dw[dw['acid_base_string'] == 'B'].index]

In [71]:
len(dw_base)

3887

In [85]:
torch.save(dw_acid, '/scratch/dz1061/jupyternotebook/pka/dataset/dw/acid/raw/temp.pt')

In [86]:
torch.save(dw_base, '/scratch/dz1061/jupyternotebook/pka/dataset/dw/base/raw/temp.pt')

In [72]:
novartis_examples = featurize(novartis, '/scratch/dz1061/jupyternotebook/pka/novartis/SMILES/')

100%|██████████| 280/280 [00:00<00:00, 761.06it/s]


In [74]:
novartis_examples[100]

{'x': tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
          0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
          0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
          0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
          0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
          0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0.,
          0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0

In [75]:
novartis_acid_examples = [novartis_examples[i] for i in novartis[novartis['acid_base_string'] == 'A'].index]

In [76]:
len(novartis_acid_examples)

95

In [77]:
novartis_base_examples = [novartis_examples[i] for i in novartis[novartis['acid_base_string'] == 'B'].index]

In [78]:
len(novartis_base_examples)

185

In [87]:
torch.save(novartis_acid_examples, '/scratch/dz1061/jupyternotebook/pka/dataset/novartis/acid/raw/temp.pt')

In [88]:
torch.save(novartis_base_examples, '/scratch/dz1061/jupyternotebook/pka/dataset/novartis/base/raw/temp.pt')

In [79]:
liter_examples = featurize(liter, '/scratch/dz1061/jupyternotebook/pka/liter/SMILES/')

100%|██████████| 123/123 [00:00<00:00, 881.53it/s]


In [80]:
liter_acid_examples = [liter_examples[i] for i in liter[liter['acid_base_string'] == 'A'].index]

In [81]:
len(liter_acid_examples)

28

In [82]:
liter_base_examples = [liter_examples[i] for i in liter[liter['acid_base_string'] == 'B'].index]

In [83]:
len(liter_base_examples)

95

In [90]:
torch.save(liter_acid_examples, '/scratch/dz1061/jupyternotebook/pka/dataset/liter/acid/raw/temp.pt')
torch.save(liter_base_examples, '/scratch/dz1061/jupyternotebook/pka/dataset/liter/base/raw/temp.pt')

In [5]:
dw.head()

,exp_id,pka,center,prot_smiles,deprot_smiles,prot_charge,deprot_charge,prot_mol,deprot_mol,acid_base_type,acid_base_check,acid_base_string,ROMol,MolWt,HeavyAtomCount,NumHDonors,NumHAcceptors,MolLogP,NumRotatableBonds,SMILES
0,mol0,6.21,10,Brc1c(N2CCCCCC2)nc(C2CC2)[nH+]c1NC1CC1,Brc1c(NC2CC2)nc(C2CC2)nc1N1CCCCCC1,1,0,<rdkit.Chem.PropertyMol.PropertyMol object at ...,<rdkit.Chem.PropertyMol.PropertyMol object at ...,basic,consistent,B,<rdkit.Chem.PropertyMol.PropertyMol object at ...,350.110609,21,1,4,4.0712,4,Brc1c(NC2CC2)nc(C2CC2)nc1N1CCCCCC1
1,mol1,7.46,9,Brc1cc(Br)c(NC2=[NH+]CCN2)c(Br)c1,Brc1cc(Br)c(NC2=NCCN2)c(Br)c1,1,0,<rdkit.Chem.PropertyMol.PropertyMol object at ...,<rdkit.Chem.PropertyMol.PropertyMol object at ...,basic,consistent,B,<rdkit.Chem.PropertyMol.PropertyMol object at ...,394.826834,15,2,3,3.3452,1,Brc1cc(Br)c(NC2=NCCN2)c(Br)c1
2,mol2,4.2,9,Brc1cc2cccnc2c2[nH+]cccc12,Brc1cc2cccnc2c2ncccc12,1,0,<rdkit.Chem.PropertyMol.PropertyMol object at ...,<rdkit.Chem.PropertyMol.PropertyMol object at ...,basic,consistent,B,<rdkit.Chem.PropertyMol.PropertyMol object at ...,257.979260,15,0,2,3.5455,0,Brc1cc2cccnc2c2ncccc12
3,mol3,3.73,8,Brc1ccc(-c2nn[nH]n2)cc1,Brc1ccc(-c2nn[n-]n2)cc1,0,-1,<rdkit.Chem.PropertyMol.PropertyMol object at ...,<rdkit.Chem.PropertyMol.PropertyMol object at ...,acidic,consistent,A,<rdkit.Chem.PropertyMol.PropertyMol object at ...,223.969758,12,1,3,1.6292,1,Brc1ccc(-c2nn[nH]n2)cc1
4,mol4,11.0,13,Brc1ccc(Br)c(N(CC2CC2)C2=[NH+]CCN2)c1,Brc1ccc(Br)c(N(CC2CC2)C2=NCCN2)c1,1,0,<rdkit.Chem.PropertyMol.PropertyMol object at ...,<rdkit.Chem.PropertyMol.PropertyMol object at ...,basic,consistent,B,<rdkit.Chem.PropertyMol.PropertyMol object at ...,370.963272,18,1,3,3.3872,3,Brc1ccc(Br)c(N(CC2CC2)C2=NCCN2)c1


In [6]:
novartis.head()

,exp_id,pka,center,prot_smiles,deprot_smiles,prot_charge,deprot_charge,prot_mol,deprot_mol,acid_base_type,acid_base_check,acid_base_string,ROMol,MolWt,HeavyAtomCount,NumHDonors,NumHAcceptors,MolLogP,NumRotatableBonds,SMILES
0,mol0,8.55,13,Brc1ccc(C2CN3C=CSC3=[NH+]2)cc1,Brc1ccc(C2CN3C=CSC3=N2)cc1,1,0,<rdkit.Chem.PropertyMol.PropertyMol object at ...,<rdkit.Chem.PropertyMol.PropertyMol object at ...,basic,consistent,B,<rdkit.Chem.PropertyMol.PropertyMol object at ...,279.966981,15,0,3,3.37980,1,Brc1ccc(C2CN3C=CSC3=N2)cc1
1,mol1,5.5,4,C/C(=C\c1cc(C)cc[nH+]1)[C@@H]1C[C@@H]2O[C@]2(C...,C/C(=C\c1cc(C)ccn1)[C@@H]1C[C@@H]2O[C@]2(C)CCC...,1,0,<rdkit.Chem.PropertyMol.PropertyMol object at ...,<rdkit.Chem.PropertyMol.PropertyMol object at ...,basic,consistent,B,<rdkit.Chem.PropertyMol.PropertyMol object at ...,501.309038,36,2,7,4.41602,2,C/C(=C\c1cc(C)ccn1)[C@@H]1C[C@@H]2O[C@]2(C)CCC...
2,mol2,4.5,5,C/C(=C\c1ccc(C)c[nH+]1)[C@@H]1C[C@@H]2O[C@]2(C...,C/C(=C\c1ccc(C)cn1)[C@@H]1C[C@@H]2O[C@]2(C)CCC...,1,0,<rdkit.Chem.PropertyMol.PropertyMol object at ...,<rdkit.Chem.PropertyMol.PropertyMol object at ...,basic,consistent,B,<rdkit.Chem.PropertyMol.PropertyMol object at ...,501.309038,36,2,7,4.41602,2,C/C(=C\c1ccc(C)cn1)[C@@H]1C[C@@H]2O[C@]2(C)CCC...
3,mol3,5.5,6,C/C(=C\c1cccc(C)[nH+]1)[C@@H]1C[C@@H]2O[C@]2(C...,C/C(=C\c1cccc(C)n1)[C@@H]1C[C@@H]2O[C@]2(C)CCC...,1,0,<rdkit.Chem.PropertyMol.PropertyMol object at ...,<rdkit.Chem.PropertyMol.PropertyMol object at ...,basic,consistent,B,<rdkit.Chem.PropertyMol.PropertyMol object at ...,501.309038,36,2,7,4.41602,2,C/C(=C\c1cccc(C)n1)[C@@H]1C[C@@H]2O[C@]2(C)CCC...
4,mol4,5.0,32,C/C(=C\c1cccc[nH+]1)[C@@H]1C[C@@H]2O[C@]2(C)CC...,C/C(=C\c1ccccn1)[C@@H]1C[C@@H]2O[C@]2(C)CCC[C@...,1,0,<rdkit.Chem.PropertyMol.PropertyMol object at ...,<rdkit.Chem.PropertyMol.PropertyMol object at ...,basic,consistent,B,<rdkit.Chem.PropertyMol.PropertyMol object at ...,487.293388,35,2,7,4.10760,2,C/C(=C\c1ccccn1)[C@@H]1C[C@@H]2O[C@]2(C)CCC[C@...


In [7]:
liter.head()

,exp_id,pka,center,prot_smiles,deprot_smiles,prot_charge,deprot_charge,prot_mol,deprot_mol,acid_base_type,acid_base_string,acid_base_check,ROMol,SMILES
0,mol0,7.42,10,C#CC[NH+](C)[C@H](C)Cc1ccc(F)cc1,C#CCN(C)[C@H](C)Cc1ccc(F)cc1,1,0,<rdkit.Chem.PropertyMol.PropertyMol object at ...,<rdkit.Chem.PropertyMol.PropertyMol object at ...,basic,B,consistent,<rdkit.Chem.PropertyMol.PropertyMol object at ...,C#CCN(C)[C@H](C)Cc1ccc(F)cc1
1,mol1,9.50,9,CC(=O)CC1CCCC[NH2+]1,CC(=O)CC1CCCCN1,1,0,<rdkit.Chem.PropertyMol.PropertyMol object at ...,<rdkit.Chem.PropertyMol.PropertyMol object at ...,basic,B,consistent,<rdkit.Chem.PropertyMol.PropertyMol object at ...,CC(=O)CC1CCCCN1
2,mol2,9.10,6,CC(=O)CCC[NH+](C)C,CC(=O)CCCN(C)C,1,0,<rdkit.Chem.PropertyMol.PropertyMol object at ...,<rdkit.Chem.PropertyMol.PropertyMol object at ...,basic,B,consistent,<rdkit.Chem.PropertyMol.PropertyMol object at ...,CC(=O)CCCN(C)C
3,mol3,3.60,8,CC(=O)OC(CC(=O)O)C[N+](C)(C)C,CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C,1,0,<rdkit.Chem.PropertyMol.PropertyMol object at ...,<rdkit.Chem.PropertyMol.PropertyMol object at ...,acidic,A,consistent,<rdkit.Chem.PropertyMol.PropertyMol object at ...,CC(=O)OC(CC(=O)O)C[N+](C)(C)C
4,mol4,9.40,8,CC(=O)OCCCC[NH+](C)C,CC(=O)OCCCCN(C)C,1,0,<rdkit.Chem.PropertyMol.PropertyMol object at ...,<rdkit.Chem.PropertyMol.PropertyMol object at ...,basic,B,consistent,<rdkit.Chem.PropertyMol.PropertyMol object at ...,CC(=O)OCCCCN(C)C


In [8]:
len(set(dw['SMILES']) & set(novartis['SMILES']))

0

In [9]:
len(set(dw['SMILES']) & set(liter['SMILES']))

0